In [56]:
import pandas as pd
from requests_html import HTMLSession

In [57]:
session = HTMLSession()
r = session.get("https://incidentreports.uchicago.edu/incidentReportArchive.php?startDate=03%2F01%2F2024&endDate=03%2F10%2F2024")
num_pages = int(r.html.search("1 / {}")[0])

In [62]:
crime_table = pd.DataFrame()
for i in range(num_pages):
    link = f"https://incidentreports.uchicago.edu/incidentReportArchive.php?startDate=03%2F01%2F2024&endDate=03%2F10%2F2024&offset={num_pages*5}"
    table = pd.read_html(link)[0]
    crime_table = pd.concat([crime_table, table], ignore_index = True)

In [63]:
crime_table

,Incident,Location,Reported,Occurred,Comments / Nature of Fire,Disposition,UCPDI#
0,Assist Other Agency,5656 S. Maryland (UCMED),3/1/24 2:08 PM,3/1/24 2:08 PM,A handgun and suspected narcotics were found a...,CPD,24-00212
1,Information / Criminal Damage to Vehicle,951 E. 61st St. (Public Way),3/2/24 9:21 AM,2/29/24 to 3/1/24 9:00 PM to 9:00 AM,Sun roof glass smashed in 2019 Infiniti parked...,CPD,2024-008280
2,Theft,1554 E. 55th St. (Walgreens),3/2/24 11:53 AM,3/2/24 11:53 AM,A subject took merchandise from the off-campus...,Ex. Cleared,24-00213
3,Found Property,5200 S. Kimbark (Public Way),3/2/24 11:25 PM,3/2/24 11:25 PM,Cell phone found on the street was turned over...,Closed,24-00214
4,Information,5815 S. Maryland (Mitchell Hospital),3/3/24 9:22 AM,2/29/24 to 3/3/24 8:55 AM to 7:40 AM,Information regarding harassing text messages ...,Closed,24-00215
5,Assist Other Agency,5656 S. Maryland (UCMED),3/1/24 2:08 PM,3/1/24 2:08 PM,A handgun and suspected narcotics were found a...,CPD,24-00212
6,Information / Criminal Damage to Vehicle,951 E. 61st St. (Public Way),3/2/24 9:21 AM,2/29/24 to 3/1/24 9:00 PM to 9:00 AM,Sun roof glass smashed in 2019 Infiniti parked...,CPD,2024-008280
7,Theft,1554 E. 55th St. (Walgreens),3/2/24 11:53 AM,3/2/24 11:53 AM,A subject took merchandise from the off-campus...,Ex. Cleared,24-00213
8,Found Property,5200 S. Kimbark (Public Way),3/2/24 11:25 PM,3/2/24 11:25 PM,Cell phone found on the street was turned over...,Closed,24-00214
9,Information,5815 S. Maryland (Mitchell Hospital),3/3/24 9:22 AM,2/29/24 to 3/3/24 8:55 AM to 7:40 AM,Information regarding harassing text messages ...,Closed,24-00215
